In [173]:
# utility functions for accessing transfermarkt.com

from urllib2 import urlopen, Request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import re
import os.path 
import datetime 

# transfermarkt blocks default useragent
useragent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
# "lol" can be anything - transfermarkt ignores it/uses it for semantic url
urlprefix = 'http://www.transfermarkt.com/lol/'

def getUrlByLeagueId(id, season=2015):
    return urlprefix + 'startseite/wettbewerb/' + id + '/saison_id/' + str(season)

def getUrlByClubId(id, season=2015):
    return urlprefix + 'startseite/verein/' + str(id) + '/saison_id/' + str(season)

def getUrlByPlayerId(id):
    return urlprefix + 'profil/spieler/' + str(id)

def getUrlByAgentId(id):
    return urlprefix + 'beraterfirma/berater/' + str(id)

def getBsByPlayerId(id):
        return BeautifulSoup(urlopen(Request(getUrlByPlayerId(id), headers={'User-Agent': useragent})))

def getClubsByLeagueId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlByLeagueId(id, season), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("td",class_="hauptlink no-border-links hide-for-small hide-for-pad")
    return [{'clubId': e.find("a")["id"], 'name': e.getText().strip(u'\xa0 ')} for e in elements]

def getPlayersByClubId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlByClubId(id, season), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("span",class_="hide-for-small")
    return [{'playerId': e.find("a", class_="spielprofil_tooltip")["id"], 
             'name': e.getText()} for e in elements if e.find("a", class_="spielprofil_tooltip")]

def getTransfersFromBs(bs):
    elements = bs.find(class_="transferhistorie").find_all("tr",class_="zeile-transfer")
    dicts = [{'seasonDate': "  ".join([td.getText() for td in e.findAll("td")[:2]]),
      'mv': e.find("td",class_="zelle-mw").getText(),
      'fee': e.find("td",class_="zelle-abloese").getText(),
      'countries': dict(zip(['from','to'],([country.find("img").get("title")
                                    for country in e.find_all("td", class_="no-border-rechts no-border-links flagge hide-for-small") if country.find("img")]))),
      'teams': dict(zip(['from','to'],([{'teamId': team.find("a")["id"], 'name': team.getText()}
                                    for team in e.find_all("td", class_="hauptlink no-border-links hide-for-small vereinsname")])))} for e in elements]
    return [{'season': d['seasonDate'].split("  ")[0],
       'date': d['seasonDate'].split("  ")[1],
       'mv': d['mv'], 'fee': d['fee'],
       'fromTeamId': d['teams']['from']['teamId'],
       'fromTeamName': d['teams']['from']['name'].lstrip(),
       'toTeamId': d['teams']['to']['teamId'],
       'toTeamName': d['teams']['to']['name'].lstrip(),
       'fromCountry': d['countries'].get('from'),
       'toCountry': d['countries'].get('to'),
        'playerName': bs.find("h1").getText()} for d in dicts]

def getPlayerDataFromBs(bs):
    elements = bs.find(class_="spielerdaten").find_all("tr") 
    result1 = [{'key': e.find("th").getText().strip().rstrip(':'),'value': e.find("td").getText().strip(), 
            'country': (e.find('img',class_="flaggenrahmen").get("title")) if e.find('img',class_="flaggenrahmen") else "",
            'hrefs' : [a.get("href") for a in e.findAll("a")]} for e in elements]

    result2 = [{'key':e['key'], 'value': (e['value'] 
           + (" COUNTRY:" + e['country'] if e['key'] == 'Place of birth' else '')
           + (" HREFs: " + " ".join(e['hrefs']) if len(e['hrefs']) > 0 else '')).strip()
          } for e in result1]

    result3 = dict([(e['key'],e['value']) for e in result2])
    
    result3["Display name"] = bs.find("h1").getText()
    print "now processing " + result3["Display name"].encode('utf-8')
    if bs.find("div",class_="detailpositionen"):
        result3["Detailed position"] = re.sub('\s+', ' ',bs.find("div",class_="detailpositionen").getText().strip())
    if bs.find("span",itemprop="birthDate"):
        result3["birthDate"] = re.sub('\s+', ' ',bs.find("span",itemprop="birthDate").getText().strip())
    result3["Date of birth"] = result3["Date of birth"].split("HREFs:")[0].strip()
    result3["Current club id"] = result3["Current club"].split("/verein/")[-1]
    result3["Current club"] = result3["Current club"].split("HREFs:")[0].strip()
    if "Outfitter" in result3:
        result3["Outfitter"] = result3["Outfitter"].split("HREFs:")[0].strip()
    if "Shoe model" in result3:
        result3["Shoe model"] = result3["Shoe model"].split("HREFs:")[0].strip()
    if "Glove" in result3:
        result3["Glove"] = result3["Glove"].split("HREFs:")[0].strip()
    if "Player's agent" in result3:
        result3["Player's agent id"] = result3["Player's agent"].split("/")[-1].strip()
        result3["Player's agent"] = result3["Player's agent"].split("HREFs:")[0].strip()
    if "Place of birth" in result3:
        result3["Country of birth"] = result3["Place of birth"].split("COUNTRY:")[-1].strip()
        result3["Place of birth"] = result3["Place of birth"].split("COUNTRY:")[0].strip()
    result3["Nationality"] = re.sub('\s+', ' ', result3["Nationality"])
    if "on loan from" in result3:
        result3["on loan from club id"] = result3["on loan from"].split("/verein/")[-1].strip()
        result3["on loan from"] = result3["on loan from"].split("HREFs:")[0].strip()
    if "2nd club" in result3:
        result3["2nd club id"] = result3["2nd club"].split("/verein/")[-1].strip()
        result3["2nd club"] = result3["2nd club"].split("HREFs:")[0].strip()
    if "Social media" in result3:    
        socialmedia = dict([(url.split("//")[-1].split("/")[0].split(".")[-2], url) for url in result3["Social media"].split(" ") if url.startswith("http")])    
        for platform in ['twitter', 'facebook', 'instagram']:
            if platform in socialmedia:
                result3[platform]=socialmedia[platform]
        if len([x for x in socialmedia if not x in ['twitter', 'facebook', 'instagram']]) > 0:
            result3['website']=socialmedia[[x for x in socialmedia if not x in ['twitter', 'facebook', 'instagram']][0]]
        del result3["Social media"]
    nationalTeamLinks=[a for a in bs.find(class_="dataContent").findAll("a") if '/nationalmannschaft/' in a.get("href")]
    if nationalTeamLinks:
        result3['Intl caps/goals'] = '/'.join([a.getText() for a in nationalTeamLinks])
        result3['National team id'] = nationalTeamLinks[0].get("href").split("/")[-1]
    
    return result3

In [167]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

In [146]:
bs = getBsByPlayerId(69651)

In [151]:
#getPlayerDataFromBs(bs)
DataFrame(getTransfersFromBs(bs))

,date,fee,fromCountry,fromTeamId,fromTeamName,mv,playerName,season,toCountry,toTeamId,toTeamName
0,"May 16, 2016",End of loan,Scotland,43,Heart of Midl.,700 Th. €,Abiola Dauda,15/16,Netherlands,499,Vitesse
1,"Feb 1, 2016",Loan,Netherlands,499,Vitesse,700 Th. €,Abiola Dauda,15/16,Scotland,43,Heart of Midl.
2,"Jul 24, 2014",500 Th. €,Serbia,159,Red Star,"1,25 Mill. €",Abiola Dauda,14/15,Netherlands,499,Vitesse
3,"Feb 8, 2013",Free transfer,Sweden,3654,Kalmar FF,"1,00 Mill. €",Abiola Dauda,12/13,Serbia,159,Red Star
4,"Jan 1, 2008",?,Sweden,8905,Sölvesborgs GoIF,-,Abiola Dauda,07/08,Sweden,3654,Kalmar FF


In [97]:
bs = BeautifulSoup(urlopen(Request(getUrlByPlayerId(7767), headers={'User-Agent': useragent})))

In [131]:
nationalTeamLinks=[a for a in bs.find(class_="dataContent").findAll("a") if '/nationalmannschaft/' in a.get("href")]

if nationalTeamLinks:
    print '/'.join([a.getText() for a in nationalTeamLinks])
    print nationalTeamLinks[0].get("href").split("/")[-1]

110/38
3375


In [175]:
index = pd.read_csv('../data/index/NL1_2015.csv', index_col=0,names=['name'])
index

,name
22372,Tom van Hyfte
37217,Pär Hansson
51969,Marcel Appiah
56379,Erik Quekel
57333,Jergé Hoefdraad
58342,Sheran Yeini
60289,Rajko Brezancic
64511,Jeroen Tesselaar
75219,Jasper Heusinkveld
78818,Dario Dumic


In [217]:
league='NL1'
season=2015
filename=league+'_'+str(season)
personalFile='../data/personal/'+filename+'.csv'
transfersFile='../data/transfers/'+filename+'.csv'
if os.path.isfile(personalFile) and os.path.isfile(transfersFile) \
    and (not 'error' in pd.read_csv(personalFile).columns) \
    and (not 'error' in pd.read_csv(transfersFile).columns):
    print str(datetime.datetime.now()), filename, "files already complete"
else:
    print str(datetime.datetime.now()), "retrieving", filename
    index = pd.read_csv('../data/index/'+league+'_'+str(season)+'.csv', index_col=0,names=['name'])
    dataList=[]
    transfersList=[]
    for id in [22372,51969,37217]:
        try:
            bs = getBsByPlayerId(id)
        except Exception as e:
            print "error", e
        try:
            data = getPlayerDataFromBs(bs)
            data.update({'id':id})
        except Exception as e:
            print "error", e
            data = {'id':id, 'error': e}
        dataList.append(data)
        try:
            transfers = getTransfersFromBs(bs)
            [transfer.update({'id':id}) for transfer in transfers]
        except Exception as e:
            transfers = [{'id':id, 'error': e}]
        transfersList.append(transfers)
DataFrame(dataList).to_csv(personalFile,index=False,encoding='utf-8')
pd.concat([DataFrame(transfers) for transfers in transfersList], ignore_index=True).to_csv(transfersFile,index=False,encoding='utf-8')
print str(datetime.datetime.now()), "done"   

2016-05-21 02:12:05.431244 NL1_2015 files already complete
2016-05-21 02:12:05.445932 done


In [213]:
'error' in pd.read_csv(transfersFile).columns

True

In [204]:
DataFrame(dataList)
#pd.concat([DataFrame(transfers) for transfers in transfersList], ignore_index=True)

,Age,Contract until,Country of birth,Current club,Current club id,Date of birth,Date of last contract extension,Detailed position,Display name,Foot,...,Name in home country,National team id,Nationality,Place of birth,Player's agent,Player's agent id,Position,birthDate,id,website
0,30,30.06.2018,Belgium,Roda JC Kerkrade,192,"Apr 28, 1986","Sep 4, 2015",Main position:Central Midfield Side position: ...,Tom van Hyfte,left,...,NaN,NaN,Belgium,Gent,Forza Sports Group,2553,Midfield - Central Midfield,"Apr 28, 1986 (30)",22372,NaN
1,29,30.06.2017,Sweden,Feyenoord Rotterdam,234,"Jun 22, 1986",NaN,Main position:Keeper,Pär Hansson,right,...,Pär Johan Åke Hansson,3557,Sweden,Vejbystrand,Global Soccer Management,786,Goalkeeper,"Jun 22, 1986 (29)",37217,NaN
2,28,30.06.2016,Germany,NEC Nijmegen,467,"Mar 26, 1988","Jun 26, 2015",Main position:Right-Back Side position: Centre...,Marcel Appiah,right,...,NaN,NaN,Germany Ghana,Schwelm,neblung sportsnetwork,175,Defence - Right-Back,"Mar 26, 1988 (28)",51969,NaN
3,29,30.06.2017,Netherlands,De Graafschap Doetinchem,642,"Apr 16, 1987",NaN,Main position:Centre Forward Side position: Ri...,Erik Quekel,NaN,...,NaN,NaN,Netherlands,Den Bosch,NaN,NaN,Striker - Centre Forward,"Apr 16, 1987 (29)",56379,NaN
4,29,30.06.2016,Netherlands,SC Cambuur-Leeuwarden,133,"Jul 17, 1986",NaN,Main position:Left Wing Side position: Attacki...,Jergé Hoefdraad,left,...,NaN,NaN,Netherlands Suriname,Amsterdam,NaN,NaN,Striker - Left Wing,"Jul 17, 1986 (29)",57333,https://www.jergehoefdraad.nl
5,29,30.06.2019,Israel,Vitesse Arnhem,499,"Dec 8, 1986",NaN,Main position:Defensive Midfield Side position...,Sheran Yeini,right,...,שרן ייני‎,5547,Israel,Tel Aviv,Top Sport Services,231,Midfield - Defensive Midfield,"Dec 8, 1986 (29)",58342,NaN
6,26,30.06.2018,Jugoslawien (SFR),AZ Alkmaar,1090,"Aug 21, 1989",NaN,Main position:Left-Back Side position: Left Wing,Rajko Brezancic,left,...,Рајко Брежанчић,9566,Serbia Bosnia-Herzegovina,Vlasenica,GET11 AG,2208,Defence - Left-Back,"Aug 21, 1989 (26)",60289,NaN


In [ ]:
new BeautifulSoup()

In [56]:
for season in range(2015,2016):
    for league in ['GB1']:
        filename = league + "_" + str(season) + ".xls"
        if os.path.isfile(filename):
            print str(datetime.datetime.now()), filename, "already exists"
        else:
            print str(datetime.datetime.now()), "retrieving", filename
            DataFrame([player for club in getClubsByLeagueId(league,season) \
                                             for player in getPlayersByClubId(club['clubId'],season)]).to_excel(filename)
            print str(datetime.datetime.now()), "done"

GB1_2015.xls already exists


In [69]:
str(datetime.datetime.now())

'2016-05-19 23:30:41.259962'

In [ ]:
#epl = [getPlayerData(player["playerId"]) for club in getClubsByLeagueId('GB1') for player in getPlayersByClubId(club["clubId"])]
bundesliga = [getPlayerData(player["playerId"]) for club in getClubsByLeagueId('L1')[:2] for player in getPlayersByClubId(club["clubId"])]
DataFrame(bundesliga).to_excel("bundesliga.xls")

In [ ]:
DataFrame(getTransfersByPlayerId(121483))

In [ ]:
DataFrame(getClubsByLeagueId('GB1',2010))  #.sort_values(by="clubId")

In [ ]:
DataFrame(getPlayersByClubId(418,2005))